In [ ]:
import matplotlib.pyplot as plt
from ipywidgets import interact, interact_manual, FloatSlider, IntSlider, IntRangeSlider, SelectMultiple, Checkbox, Dropdown, HBox, VBox, interactive_output, Label, Layout
from IPython.display import display

import sys
sys.path.append('py')
import fnr as fnr

In [ ]:
ind = ['23101', '23102', '23103', '23104', '23105', '23106', '23107', '23108', '23109', '23110', '23120', '23130', '23140', '23150',
       '23160', '23170', '23180', '23190', '23201', '23207', '23208', '23210', '23220', '23231', '23232', '23235', '23236', '23241',
       '23243', '23244', '23245', '23251', '23259', '23260', '23270', '23280', '23290', '23301', '23302', '23309', '23310', '23320',
       '23331', '23332']

fnr = fnr.fnr_class(2008, 2020, {'lists': ['pubagg', 'a38np'], 'mappings': {'23IND': ind}})
fnr.df

In [ ]:
fnr.return_aggregate('2X29_30', 'VLP')

In [ ]:
def plott_aggregat(df):
    aargang_min = df.index.get_level_values('årgang').min().year
    aargang_max = df.index.get_level_values('årgang').max().year
    variabler = [x for x in df.index.get_level_values('nr_variabler').unique().to_list() if x != 'BNPF']
    aggregeringer = df.index.get_level_values('aggregering').unique().to_list()
    regionsreformer = ['-2017', '2018-2019', '2020-']

    def lag_aggregater(variabel, aggregering):
        if variabel == 'BRIN':
            return [x for x in df[df.index.get_level_values('aggregering') == aggregering].index.get_level_values('aggregat').unique().to_list() if x[0] == '8']
        else:
            return [x for x in df[df.index.get_level_values('aggregering') == aggregering].index.get_level_values('aggregat').unique().to_list() if x[0] == '2']

    def lag_fylker(regionsreform):
        if regionsreform == '-2017':
            return ['f01', 'f02', 'f03', 'f04', 'f05', 'f06', 'f07', 'f08', 'f09', 'f10', 'f11', 'f12', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19', 'f20', 'f21', 'f23']
        if regionsreform == '2018-2019':
            return ['f01', 'f02', 'f03', 'f04', 'f05', 'f06', 'f07', 'f08', 'f09', 'f10', 'f11', 'f12', 'f14', 'f15', 'f50', 'f18', 'f19', 'f20', 'f21', 'f23']
        if regionsreform == '2020-':
            return ['f30', 'f03', 'f34', 'f38', 'f42', 'f11', 'f46', 'f15', 'f50', 'f18', 'f54', 'f21', 'f23']

    storrelse_wgt = FloatSlider(value=10, min=0, max=20)
    aargang_wgt = IntRangeSlider(value=[aargang_min, aargang_max], min=aargang_min, max=aargang_max)
    variabel_wgt = Dropdown(options=variabler, description = 'Variabel', style={'description_width': '3cm'})
    aggregering_wgt = Dropdown(options=aggregeringer, value=aggregeringer[1], description = 'Aggregering', style={'description_width': '3cm'})
    aggregat_wgt = Dropdown(options=lag_aggregater(variabler[0], aggregeringer[1]), description = 'Aggregat', style={'description_width': '3cm'})
    regionsreformer_wgt = Dropdown(options=regionsreformer, value=regionsreformer[2], description = 'Regionsreform', style={'description_width': '3cm'})
    fylker_wgt = SelectMultiple(options=lag_fylker(regionsreformer[2]), rows=20)
    stack_wgt = Checkbox(value=True, description = 'Stable stolper')

    def oppdater_aggregater(arg):
        aggregat_wgt.options = lag_aggregater(variabel_wgt.value, aggregering_wgt.value)

    def oppdater_fylker(arg):
        fylker_wgt.options = lag_fylker(regionsreformer_wgt.value)

    variabel_wgt.observe(oppdater_aggregater)
    aggregering_wgt.observe(oppdater_aggregater)
    regionsreformer_wgt.observe(oppdater_fylker)

    def plot_naering(aargang, aggregering, aggregat, variabel, fylker, stack, storrelse):
        try:
            betingelse = (
                (df.index.get_level_values('aggregering') == aggregering) &
                (df.index.get_level_values('aggregat') == aggregat) &
                (df.index.get_level_values('nr_variabler') == variabel) &
                (df.index.get_level_values('årgang').year >= aargang[0]) &
                (df.index.get_level_values('årgang').year <= aargang[1])
            )
            fnr.df[betingelse][list(fylker)].reset_index().set_index('årgang').plot.bar(figsize=(storrelse, storrelse/2), stacked=stack)
            plt.xlabel('Årgang')
            plt.legend(ncol=10)
            plt.show()
        except TypeError:
            pass

    ui = VBox([aargang_wgt, variabel_wgt, aggregering_wgt, aggregat_wgt, regionsreformer_wgt, fylker_wgt, stack_wgt, storrelse_wgt])
    out = interactive_output(plot_naering, {'aargang': aargang_wgt,
                                            'aggregering': aggregering_wgt,
                                            'aggregat': aggregat_wgt,
                                            'variabel': variabel_wgt,
                                            'fylker': fylker_wgt,
                                            'stack': stack_wgt,
                                            'storrelse': storrelse_wgt})
    display(HBox([ui, out]))


plott_aggregat(fnr.df)